Notebook 4: filtering to common independent SNPS, relatedness, PCA, joint calling with new dataset, applying RF on new dataset. Analyses that need to be run:
1. Work with Lindo: - *PENDING*
    - Joint calling with GGV, sample QC 
    - Use gnomAD RF (subset to variants in RF model) - doesn’t need VQSR 
    - Intersect HGDP+1kG+GGV, build a RF with 1kG+ HGDP, apply it to a new dataset (GGV) - doesn’t need VQSR 
2. PCA plots - Ally - *PENDING*
    - already implemented in R, just need to plot it in Hail
    
----------------------------------------
Further edits needed in this nb: 
- Update the global and subcontinental paths and add them to section 1
- Add the path to the PCA plotting Rmarkdown (in section 5) once available  
- Complete the table in section 5
- Add Ally's code for plots 

# Index
1. [Setting Default Output Paths](#1.-Set-Default-Output-Paths)
2. [Variant Filtering and LD Pruning](#2.-Variant-Filtering-and-LD-Pruning)
3. [Run PC Relate](#3.-Run-PC-Relate)
4. [PCA](#4.-PCA)
    1. [Function to Run PCA on Unrelated Individuals](#4a.-Function-to-Run-PCA-on-Unrelated-Individuals)
    2. [Function to Project Related Individuals](#4b.-Function-to-Project-Related-Individuals)
    3. [Global PCA](#4c.-Global-PCA)
    4. [Subcontinental PCA](#4d.-Subcontinental-PCA)
5. [Outlier Removal](#5.-Outlier-Removal)
6. [Rerun PCA](#6.-Rerun-PCA)
    1. [Global PCA](#6a.-Global-PCA)
    2. [Subcontinental PCA](#6b.-Subcontinental-PCA)
7. [Writing out Matrix Table](#7.-Write-Out-Matrix-Table)

# General Overview 
The purpose of this notebook is to further filter the postQC matrix table to prepare it for LD pruning, run relatedness and Principal Component Analysis (PCA), joint call with new data set, and apply RF. 

**This script contains information on how to:**
- Read in the a matrix table and run Hail common variant statistics  
- Filter using allele frequency and call rate
- Run LD pruning 
- Run relatedness and separate related and unrelated individuals
- Calculate PC scores and project samples on to a PC space  
- Run global and Subcontinental PCA and plot them 
- Remove PCA outliers (filter using sample IDs)
- Joint call with a new data set
- Build/apply RF
- Write out a matrix table 

Author: Mary T. Yohannes

In [ ]:
# import hail
import hail as hl

# import the read_qc function
from read_qc_function import read_qc

# importing methods from gnomAD needed to project individuals
from gnomad.sample_qc.ancestry import *

## Set Requester Pays Bucket
Running through these tutorials, users must specify which project is to be billed. To change which project is billed, set the `GCP_PROJECT_NAME` variable to your own project.

In [ ]:
# setting requester pays bucket to use throughout tutorial
GCP_PROJECT_NAME = "diverse-pop-seq-ref"
hl.init(spark_conf={
    'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
    'spark.hadoop.fs.gs.requester.pays.buckets': 'hgdp_tgp,gcp-public-data--gnomad',
    'spark.hadoop.fs.gs.requester.pays.project.id': 'diverse-pop-seq-ref'
})

# 1. Set Default Output Paths
These default paths can be edited by users as needed. It is recommended to run these tutorials without writing out datasets. The read_qc() function is intended to take the place of needing to write out and read in datasets by the user. 

By default we have commented out all of the write steps of the tutorials, if you would like to write out your own datasets, uncomment those sections and replace the paths with your own. 

In [ ]:
# input 
input_path = 'gs://hgdp-1kg/hgdp_tgp/intermediate_files/pre_running_varqc.mt'

# intermediate file
intermediate_path = 'gs://hgdp-1kg/hgdp_tgp/intermediate_files/filtered_n_pruned_output_updated.mt'

# pre-outlier paths for unrelated and related samples 
unrel_path = 'gs://hgdp-1kg/hgdp_tgp/unrel_updated.mt'
rel_path = 'gs://hgdp-1kg/hgdp_tgp/rel_updated.mt' 

# pre-outlier file path is missing - global & subcont pca results [here]

# outliers file 
outliers_path = 'gs://hgdp-1kg/hgdp_tgp/pca_outliers_v2.txt'

# post-outlier file path is missing - global & subcont pca results[here]

# final output paths for unrelated and related samples (post-outlier)
unrel_output = 'gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/unrelated.mt'
rel_output = 'gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/related.mt'

# 2. Variant Filtering and LD Pruning
<br>
<details><summary> <b>Why are we doing this?</b>
    <br>[click <span style="color:green">here</span> to expand] </summary>
    
At this point, we have 155,648,020 SNPs and since we need fewer number of variants (~100-300k) for PCA, we filter on:
- AF - allele frequency 
- call rate - fraction of calls neither missing nor filtered

and then run LD pruning.     
    
Linkage disequilibrium (LD) is the correlation between nearby variants such that the alleles at neighboring polymorphisms (observed on the same chromosome) are associated within a population more often than if they were unlinked.
<br>    
For more information on LD pruning click <a href=""> here </a>
</details>

<br>
<details><summary> <b>More information on Hail methods and expressions</b> 
    <br>[click <span style="color:green">here</span> to expand] </summary>
<ul>
<li><a href="https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc"> More on  <i> variant_qc() </i></a></li>

<li><a href="https://hail.is/docs/0.2/methods/genetics.html#hail.methods.ld_prune"> More on  <i> ld_prune() </i></a></li>
    </ul>
</details>

[Back to Index](#Index)

In [ ]:
# read-in the right intermediate file 
mt_filt = hl.read_matrix_table(input_path)

#### 2a. Variant Filtering 

In [ ]:
# run Hail's common variant statistics (QC metrics) 
mt_var = hl.variant_qc(mt_filt) 

# filter to variants with AF between 0.05 & 0.95, and call rate greater than 0.999    
mt_var_filt = mt_var.filter_rows((mt_var.variant_qc.AF[0] > 0.05) & 
                                 (mt_var.variant_qc.AF[0] < 0.95) & (mt_var.variant_qc.call_rate > 0.999))

# Should print 6787034 snps; this line take ~20min to run 
print('Num of variants after filtering = ' + str(mt_var_filt.count()[0]))

#### 2b. LD Pruning
Since the number of variants after this step is now in the ~100-300k range, we proceed to the PCA analysis without any more adjustments.  

In [ ]:
# remove correlated variants 
pruned = hl.ld_prune(mt_var_filt.GT, r2=0.1, bp_window_size=500000) # ~113 min to run  
mt_var_pru_filt = mt_var_filt.filter_rows(hl.is_defined(pruned[mt_var_filt.row_key])) 
print('Num of variants after LD pruning = ' + str(mt_var_pru_filt.count()[0])) # 248634 snps

#### 2c. Write out an intermediate file
The LD pruning step takes a non negligble time to run so to ensure that the downstream analyses steps don't take a very long time we write out an intermediate file. This write out step should take around 23 minutes to run. 

Due to the use of the read_qc function however, you do not need to run through the write out step. Instead, the function will automatically read in the version of the dataset we wrote out when creating these tutorials. 

If the user wishes to export their own intermediate file, they can do so by changing the intermediate path and then replacing the read_qc() function call with `hl.read_matrix_table(intermediate_path)`

In [ ]:
# # writing out an intermediate file to speed up subsequent analyses
# mt_var_pru_filt.write(intermediate_path, overwrite=False)

# read the intermediate file back in for subsequent analyses
mt_var_pru_filt = read_qc(ld_prune=True)

# 3. Run PC Relate   
<br>
<details><summary> <b>Why are we doing this?</b>
    <br>[click <span style="color:green">here</span> to expand] </summary>
<br>
When doing Principal Component Analysis (PCA), we need to separate the related and unrelated samples before computing the PC scores and ploting them. This is because if we compute PCA with the related samples in the data set, the population structure and clustering will be affected by the relatedness that exists among those samples. Thus, we first have to identify the related individuals by computing relatedness estimates (kinship statistic in this case) using a variant of the PC-Relate method in Hail. We used a minimum minor allele frequency (MAF) filter of 0.05, excluded sample pairs with kinship less than 0.05, and used 20 principal components (PC) to control for population structure. After getting the sample ID pairs for the related samples, we then separate the filtered and pruned mt into relateds and unrelateds.
<br>    
For more information on relatedness click <a href="https://hail.is/docs/0.2/methods/relatedness.html#relatedness"> here</a>
    
</details>

<br>
<details><summary> <b>What metrics are we using for pc_relate?</b> 
    <br>[click <span style="color:green">here</span> to expand] </summary>
<br>
We computed the kinship statistic using:
<ul>
<li>a minimum minor allele frequency filter of 0.05</li>
<li>excluding sample-pairs with kinship less than 0.05</li>
<li>20 principal components to control for population structure</li>
</ul>
    
For more information on the pc_relate method click <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4716688/">here</a>
    
</details>

<br>
<details><summary> <b>More information on Hail methods and expressions</b> 
    <br>[click <span style="color:green">here</span> to expand] </summary>
<ul>
<li><a href="https://hail.is/docs/0.2/methods/relatedness.html#hail.methods.pc_relate"> More on  <i> pc_relate() </i></a></li>

<li><a href="https://hail.is/docs/0.2/methods/misc.html#hail.methods.maximal_independent_set"> More on  <i> maximal_independent_set() </i></a></li>
</ul>
    
</details>

[Back to Index](#Index)

In [ ]:
# compute kinship statistic
# takes ~4min to run
relatedness_ht = hl.pc_relate(
    mt_var_pru_filt.GT, min_individual_maf=0.05, min_kinship=0.05, statistics='kin', k=20).key_by() 

# identify closely related individuals in pairs (list of sample IDs) 
# takes ~2hr & 22min to run
related_samples_to_remove = hl.maximal_independent_set(relatedness_ht.i, relatedness_ht.j, False) 

# subset the filtered and pruned mt to samples that are NOT present in the list of related individuals  
mt_unrel = mt_var_pru_filt.filter_cols(hl.is_defined(related_samples_to_remove[mt_var_pru_filt.col_key]), keep=False) 

# do the same as above but this time subset to samples that are present in the related-individuals list   
mt_rel = mt_var_pru_filt.filter_cols(hl.is_defined(related_samples_to_remove[mt_var_pru_filt.col_key]), keep=True) 

In [ ]:
# # write out the unrelated and related mts since they are used beyond this notebook in other analyses     
# # unrelated mt
# mt_unrel.write(unrel_path, overwrite=False) 

# # related mt 
# mt_rel.write(rel_path, overwrite=False)

In [ ]:
# read the related and unrelated mts back in using read_qc
# unrelated mt
mt_unrel = read_qc(rel_unrel='unrelated_pre_outlier') 

# related mt 
mt_rel = read_qc(rel_unrel='related_pre_outlier')

# 4. PCA
<br>
<details><summary> <b>What are we doing here?</b> 
    <br>[click <span style="color:green">here</span> to expand] </summary>
<br>
PCA is ran on the unrelated samples first. Then, the related samples are projected onto the PC space of the unrelated samples to get their PC scores. This way the population structure and clustering is not affected by the relatedness among samples.  
    
</details>

[Back to Index](#Index)

### 4a. Function to Run PCA on Unrelated Individuals

[Back to Index](#Index)

In [ ]:
def run_pca(mt: hl.MatrixTable, reg_name:str, out_prefix: str, overwrite: bool = False):
    """
    Runs PCA on a data set
    :param mt: data set to run PCA on
    :param reg_name: region name for saving output purposes
    :param out_prefix: path for where to save the outputs
    :return:
    """

    pca_evals, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, k=20, compute_loadings=True)
    pca_mt = mt.annotate_rows(pca_af=hl.agg.mean(mt.GT.n_alt_alleles()) / 2)
    pca_loadings = pca_loadings.annotate(pca_af=pca_mt.rows()[pca_loadings.key].pca_af)
    pca_scores = pca_scores.transmute(**{f'PC{i}': pca_scores.scores[i - 1] for i in range(1, 21)})
    
    pca_scores.export(out_prefix + reg_name + '_scores.txt.bgz')  # save individual-level-genetic-region PCs
    pca_loadings.write(out_prefix + reg_name + '_loadings.ht', overwrite)  # save PCA loadings

### 4b. Function to Project Related Individuals
<br>
<details><summary> <b>Troubleshooting:</b> 
    <br>[click <span style="color:green">here</span> to expand] </summary>
    If this function is not working, make sure you used the  <code>--packages gnomad</code> argument when starting your cluster
</details>

[Back to Index](#Index)

In [ ]:
from gnomad.sample_qc.ancestry import *

def project_individuals(pca_loadings, project_mt, reg_name:str, out_prefix: str, overwrite: bool = False):
    """
    Project samples into predefined PCA space
    :param pca_loadings: existing PCA space of unrelated samples 
    :param project_mt: matrix table of related samples to project  
    :param reg_name: region name for saving output purposes
    :param project_prefix: path for where to save PCA projection outputs
    :return:
    """
    ht_projections = pc_project(project_mt, pca_loadings)  
    ht_projections = ht_projections.transmute(**{f'PC{i}': ht_projections.scores[i - 1] for i in range(1, 21)}) 
    ht_projections.export(out_prefix + reg_name + '_projected_scores.txt.bgz') # save output   

ModuleNotFoundError: No module named 'gnomad'

### 4c. Global PCA

<br>
<details><summary> <b>Why are we doing this?</b>
    <br>[click <span style="color:green">here</span> to expand] </summary>
<br>
To see the population structure and clustering on a continental level and contextualize the data globally.    
    
</details>

[Back to Index](#Index)

In [ ]:
# run PCA on the unrelated samples
run_pca(mt_unrel, 'global', 'gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/', False)  

# read in the PCA loadings of the unrelated samples
loadings = hl.read_table('gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/global_loadings.ht') 

# project the related samples onto the unrelated-samples' PC space 
project_individuals(loadings, mt_rel, 'global', 'gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/', False) 

### 4d. Subcontinental PCA 
<br>

When running the following section, the notebook might freeze after printing the log for EUR, AFR and AMR. If this happens, do not restart it. Let it run and follow the progress with the outputs being generated.  

When complete, check for the following in your specified output path:
- 21 total output files (3 for each region)

Once you have confirmed you have the desired output do the following:
1. Save close and halt the current notebook
2. Open a new session
3. Proceed to the next step (run project_relateds function)


<br>
<details><summary> <b>Why are we doing this?</b>
    <br>[click <span style="color:green">here</span> to expand] </summary>
<br>
To see the population structure and clustering on a subcontinental level and contextualize data within continental regions. This also helped us identify outliers which were removed later on.     
</details>
<br>

<details><summary> <b>More information on Hail methods and expressions</b>
    <br>[click <span style="color:green">here</span> to expand]</summary> 

<ul>
<li><a href="more info https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca"> More on <i> hwe_normalized_pca() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_rows"> More on <i> annotate_rows() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.annotate"> More on <i> annotate() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.transmute"> More on <i> transmute() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.export"> More on <i> export() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/experimental/index.html#hail.experimental.pc_project"> More on <i> pc_project() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.expr.Expression.html#hail.expr.Expression.collect"> More on <i> collect() </i></a></li>
</ul>
    
</details>

[Back to Index](#Index)

In [ ]:
# obtain a list of the continental regions in the data set (used the unrelated data set since it had more samples) 
regions = mt_unrel['hgdp_tgp_meta']['Genetic']['region'].collect()
regions = list(dict.fromkeys(regions)) # convert into a list
# There are 7 regions: EUR, AFR, AMR, EAS, CSA, OCE, and MID

# set argument values for PCA 
subcont_pca_prefix = 'gs://hgdp-1kg/hgdp_tgp/pca_preoutlier/subcont_pca/subcont_pca_' # path for outputs 
overwrite = False

# for each region, run PCA on the unrelated samples (~27min to run)
for i in regions:  
    # filter the unrelateds per region
    subcont_unrel = mt_unrel.filter_cols(mt_unrel['hgdp_tgp_meta']['Genetic']['region'] == i)  
    run_pca(subcont_unrel, i, subcont_pca_prefix, overwrite)

In [ ]:
# for each region, project the related samples onto the unrelated-samples' PC space (~2min to run)
for i in regions:
    # read in the PCA loadings of the unrelated samples for each region 
    loadings = hl.read_table(subcont_pca_prefix + i + '_loadings.ht') 
    
    # filter the related mt per region 
    subcont_rel = mt_rel.filter_cols(mt_rel['hgdp_tgp_meta']['Genetic']['region'] == i)  
    
    # project 
    project_individuals(loadings, subcont_rel, i, subcont_pca_prefix, overwrite) 

# 5. Outlier Removal

After plotting the PCs using R (link_the_plotting_Rmarkdown_here), 22 outliers were identified (complete_the_table)

| s | Genetic region | Population | Note |
| --- | --- | --- | -- |
| NA20314 | AFR | ASW | Clusters with AMR in global PCA | 
| NA20299 | - | - | - |
| NA20274 | - | - | - |
| HG01880 | - | - | - |
| HG01881 | - | - | - |
| HG01628 | - | - | - |
| HG01629 | - | - | - |
| HG01630 | - | - | - |
| HG01694 | - | - | - |
| HG01696 | - | - | - |
| HGDP00013 | - | - | - |
| HGDP00150 | - | - | - |
| HGDP00029 | - | - | - |
| HGDP01298 | - | - | - |
| HGDP00130 | CSA | Makrani | Closer to AFR than most CSA |
| HGDP01303 | - | - | - |
| HGDP01300 | - | - | - |
| HGDP00621 | MID | Bedouin | Closer to AFR than most MID |
| HGDP01270 | MID | Mozabite | Closer to AFR than most MID |
| HGDP01271 | MID | Mozabite | Closer to AFR than most MID |
| HGDP00057 | - | - | - | 
| LP6005443-DNA_B02 | - | - | - |

<details><summary> <b>More information on Hail methods and expressions</b>
    <br>[click <span style="color:green">here</span> to expand] </summary> 

<ul>
<li><a href="more info https://hail.is/docs/0.2/utils/index.html#hail.utils.hadoop_open"> More on <i> hl.utils.hadoop_open() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/functions/core.html#hail.expr.functions.literal"> More on <i> hl.literal() </i></a></li>
    </ul>    

</details>

[Back to Index](#Index)

In [ ]:
# read in the unrelated and related mts to remove outliers and rerun pca  
mt_unrel_unfiltered = read_qc(rel_unrel='unrelated_pre_outlier') # unrelated mt
mt_rel_unfiltered = read_qc(rel_unrel='related_pre_outlier') # related mt

# read the outliers file into a list
with hl.utils.hadoop_open(outliers_path) as file: 
    outliers = [line.rstrip('\n') for line in file]
    
# capture and broadcast the list as an expression
outliers_list = hl.literal(outliers)

# remove the 22 outliers from both mts
mt_unrel = mt_unrel_unfiltered.filter_cols(~outliers_list.contains(mt_unrel_unfiltered['s']))
mt_rel = mt_rel_unfiltered.filter_cols(~outliers_list.contains(mt_rel_unfiltered['s']))

# sanity check 
print('Unrelated: Before outlier removal ' + 
      str(mt_unrel_unfiltered.count()[1]) + ' | After outlier removal ' + 
      str(mt_unrel.count()[1]))

print('Related: Before outlier removal: ' + 
      str(mt_rel_unfiltered.count()[1]) + ' | After outlier removal ' + 
      str(mt_rel.count()[1])) num_outliers = (mt_unrel_unfiltered.count()[1] - 
                                              mt_unrel.count()[1]) + (mt_rel_unfiltered.count()[1] - 
                                                                      mt_rel.count()[1])
print('Total samples removed = ' + str(num_outliers))

# 6. Rerun PCA

Before running the sections below make sure you have run sections 4a (PCA) and 4b (projection) above.

<br>
<details><summary> <b>What is different from the prior PCA run?</b>
    <br>[click <span style="color:green">here</span> to expand] </summary>
<ul>
<li>updated unrelated and related mts (outliers removed)</li>
<li>new paths for the outputs</li>  
    </ul>
</details>

[Back to Index](#Index)

### 6a. Global PCA (without outliers)

In [ ]:
# run PCA on the unrelated samples  
run_pca(mt_unrel, 'global', 'gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/', False)

# read in the PCA loadings of the unrelated samples  
loadings = hl.read_table('gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/global_loadings.ht') 

# project the related samples onto the unrelated-samples' PC space 
project_individuals(loadings, mt_rel, 'global', 'gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/', False) 

### 6b. Subcontinental PCA (without outliers)

<br>
When running the following section, the notebook might freeze after printing the log for EUR, AFR and AMR. If this happens, do not restart it. Let it run and follow the progress with the outputs being generated.  

When complete, check for the following in your specified output path:
- 21 total output files (3 for each region)

Once you have confirmed you have the desired output do the following:
1. Save close and halt the current notebook
2. Open a new session
3. Proceed to the next step (run project_relateds function)


<br>
<details><summary> <b>More information on Hail methods and expressions</b>
    <br>[click <span style="color:green">here</span> to expand] </summary> 

<ul>
<li><a href="more info https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca"> More on <i> hwe_normalized_pca() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_rows"> More on <i> annotate_rows() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.annotate"> More on <i> annotate() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.transmute"> More on <i> transmute() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.Table.html#hail.Table.export"> More on <i> export() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/experimental/index.html#hail.experimental.pc_project"> More on <i> pc_project() </i></a></li>
    
<li><a href="more info https://hail.is/docs/0.2/hail.expr.Expression.html#hail.expr.Expression.collect"> More on <i> collect() </i></a></li>
    </ul>
    
</details>

[Back to Index](#Index)

In [ ]:
# set argument values for PCA 
subcont_pca_prefix = 'gs://hgdp-1kg/hgdp_tgp/pca_postoutlier/subcont_pca/subcont_pca_' # path for outputs 
overwrite = False 


# for each region, run PCA on the unrelated samples (~26 min to run) 
# "regions" is a list containing the 7 continental regions in the data set from section 4d
for i in regions: 
    # filter the unrelateds per region
    subcont_unrel = mt_unrel.filter_cols(mt_unrel['hgdp_tgp_meta']['Genetic']['region'] == i)  
    run_pca(subcont_unrel, i, subcont_pca_prefix, overwrite)

In [ ]:
# for each region, project the related samples onto the unrelated-samples' PC space (~3min to run)
for i in regions:
    # read in the PCA loadings of the unrelated samples for each region
    loadings = hl.read_table(subcont_pca_prefix + i + '_loadings.ht') 
    
    # filter the relateds per region 
    subcont_rel = mt_rel.filter_cols(mt_rel['hgdp_tgp_meta']['Genetic']['region'] == i)  
    
    # project 
    project_individuals(loadings, subcont_rel, i, subcont_pca_prefix, overwrite) 

# 7. Write Out Matrix Table 
[Back to Index](#Index)

In [ ]:
# # write out mts of unrelated and related samples separately (post-outlier removal) 
# #unrelated mt
# mt_unrel.write('gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/unrelated.mt',
#                overwrite=False)
# #related mt
# mt_rel.write('gs://hgdp-1kg/hgdp_tgp/datasets_for_others/lindo/ds_without_outliers/related.mt',
#              overwrite=False)